## This notebook is to do missing data imputation for trainset

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option('display.max_rows',2000)
pd.set_option('display.max_columns',50)
pd.set_option('display.width',1000)

from pprint import pprint
from feature_engine import missing_data_imputers as mdi

import warnings
warnings.filterwarnings("ignore")

### Load Data and Split

In [2]:
df = pd.read_csv('train_values.csv')

In [3]:
#df

In [4]:
df.head()

,row_id,county_code,state,population,renter_occupied_households,pct_renter_occupied,evictions,rent_burden,pct_white,pct_af_am,pct_hispanic,pct_am_ind,pct_asian,pct_nh_pi,pct_multiple,pct_other,poverty_rate,rucc,urban_influence,economic_typology,pct_civilian_labor,pct_unemployment,pct_uninsured_adults,pct_uninsured_children,pct_adult_obesity,pct_adult_smoking,pct_diabetes,pct_low_birthweight,pct_excessive_drinking,pct_physical_inactivity,air_pollution_particulate_matter_value,homicides_per_100k,motor_vehicle_crash_deaths_per_100k,heart_disease_mortality_per_100k,pop_per_dentist,pop_per_primary_care_physician,pct_female,pct_below_18_years_of_age,pct_aged_65_years_and_older,pct_adults_less_than_a_high_school_diploma,pct_adults_with_high_school_diploma,pct_adults_with_some_college,pct_adults_bachelors_or_higher,birth_rate_per_1k,death_rate_per_1k
0,0,8e686a7,fb8cab1,3876.0,408.0,24.583,NaN,18.380,0.945945,0.010761,0.026038,0.005685,0.005635,0.0,0.005935,0.000000,4.172,"Nonmetro - Completely rural or less than 2,500...",Noncore adjacent to micro area and does not co...,Manufacturing-dependent,0.550,0.023,0.107,0.062,0.310,0.166,0.100,NaN,0.262,0.342,11.022908,NaN,NaN,217,NaN,NaN,0.471,0.218,0.190,0.083250,0.327984,0.389168,0.199599,10.008982,9.752342
1,1,d1b5fc5,842bd12,10224.0,1166.0,28.346,3.0,26.694,0.808959,0.002284,0.173426,0.001803,0.001493,0.0,0.012033,0.000000,7.714,"Nonmetro - Completely rural or less than 2,500...",Micropolitan not adjacent to a metro area,Recreation,0.545,0.040,0.324,0.201,0.253,0.102,0.066,0.070,0.200,0.148,9.904099,NaN,18.26,181,2009.0,1690.0,0.472,0.290,0.077,0.099099,0.202202,0.302302,0.396396,14.482893,3.013219
2,2,19a463b,2b7da97,27023.0,2927.0,21.641,9.0,31.028,0.956621,0.005141,0.013088,0.004728,0.004708,0.0,0.014809,0.000905,9.789,"Nonmetro - Urban population of 2,500 to 19,999...",Noncore not adjacent to a metro/micro area and...,Recreation,0.508,0.063,0.129,0.032,0.296,0.215,0.087,0.070,0.187,0.258,11.011502,NaN,19.94,234,4650.0,1350.0,0.496,0.208,0.192,0.137725,0.387226,0.255489,0.219561,9.620346,10.873572
3,3,1711ab7,5029ed4,8735.0,1039.0,23.110,0.0,27.734,0.894835,0.000298,0.076576,0.000995,0.004297,0.0,0.022999,0.000000,13.735,"Nonmetro - Urban population of 2,500 to 19,999...",Noncore not adjacent to a metro/micro area and...,Nonspecialized,0.545,0.038,0.207,0.084,0.255,0.182,0.104,0.081,0.169,0.274,12.845770,NaN,30.31,231,4420.0,1749.0,0.503,0.193,0.215,0.077154,0.308617,0.398798,0.215431,9.383225,10.870809
4,4,1eb4681,b795815,3681.0,365.0,21.985,2.0,19.673,0.923886,0.000496,0.042889,0.000000,0.013622,0.0,0.016905,0.002202,8.217,"Nonmetro - Completely rural or less than 2,500...",Noncore adjacent to micro area and does not co...,Farm-dependent,0.474,0.034,0.177,0.075,0.324,0.118,0.113,NaN,0.178,0.340,11.565750,NaN,NaN,270,NaN,1249.0,0.516,0.222,0.230,0.092537,0.362189,0.346269,0.199005,9.543286,14.723926


In [5]:
df.tail()

,row_id,county_code,state,population,renter_occupied_households,pct_renter_occupied,evictions,rent_burden,pct_white,pct_af_am,pct_hispanic,pct_am_ind,pct_asian,pct_nh_pi,pct_multiple,pct_other,poverty_rate,rucc,urban_influence,economic_typology,pct_civilian_labor,pct_unemployment,pct_uninsured_adults,pct_uninsured_children,pct_adult_obesity,pct_adult_smoking,pct_diabetes,pct_low_birthweight,pct_excessive_drinking,pct_physical_inactivity,air_pollution_particulate_matter_value,homicides_per_100k,motor_vehicle_crash_deaths_per_100k,heart_disease_mortality_per_100k,pop_per_dentist,pop_per_primary_care_physician,pct_female,pct_below_18_years_of_age,pct_aged_65_years_and_older,pct_adults_less_than_a_high_school_diploma,pct_adults_with_high_school_diploma,pct_adults_with_some_college,pct_adults_bachelors_or_higher,birth_rate_per_1k,death_rate_per_1k
1557,1557,8a15c79,a952566,18983.0,1480.0,22.621,41.0,26.905,0.847652,0.064786,0.048670,0.000000,0.012277,0.001800,0.024815,0.000000,5.699,Metro - Counties in metro areas of fewer than ...,Small-in a metro area with fewer than 1 millio...,Nonspecialized,0.517,0.044,0.216,0.086,0.302,NaN,0.096,0.079,NaN,0.227,12.942281,NaN,28.94,207,6299.0,4660.0,0.510,0.243,0.144,0.153614,0.347390,0.263052,0.235944,11.369618,7.211285
1558,1558,ffaffbd,e2f94fa,18837.0,2144.0,28.649,7.0,26.515,0.925500,0.010666,0.028790,0.000398,0.020893,0.001085,0.012667,0.000000,6.789,"Nonmetro - Urban population of 2,500 to 19,999...",Noncore adjacent to micro area and contains a ...,Nonspecialized,0.559,0.043,0.112,0.038,0.294,0.169,0.081,0.060,0.166,0.242,10.669456,NaN,18.02,272,2030.0,1249.0,0.510,0.205,0.193,0.070858,0.371257,0.311377,0.246507,9.330009,12.600874
1559,1559,4268c79,a952566,77224.0,4677.0,18.928,137.0,28.827,0.883936,0.070858,0.019081,0.000614,0.012569,0.000302,0.011835,0.000805,6.294,"Metro - Counties in metro areas of 250,000 to ...",Small-in a metro area with fewer than 1 millio...,Nonspecialized,0.498,0.051,0.172,0.071,0.260,0.193,0.096,0.080,0.168,0.298,12.962100,NaN,13.78,237,6979.0,1529.0,0.503,0.211,0.179,0.118644,0.311067,0.302094,0.268195,9.029286,8.846613
1560,1560,ff00193,4cd9667,4698.0,402.0,20.107,1.0,48.980,0.173909,0.000911,0.824579,0.000401,0.000000,0.000000,0.000000,0.000200,17.300,"Nonmetro - Completely rural or less than 2,500...",Noncore adjacent to micro area and does not co...,Federal/State government-dependent,0.493,0.112,0.273,0.129,0.235,0.171,0.099,0.110,0.131,0.246,10.244994,NaN,NaN,192,4710.0,NaN,0.484,0.195,0.206,0.164000,0.354000,0.342000,0.140000,8.445214,9.744478
1561,1561,38ccdfd,1dcfd4e,650813.0,92680.0,37.789,NaN,30.687,0.704111,0.057262,0.161265,0.003829,0.027315,0.003196,0.041223,0.001799,8.343,"Metro - Counties in metro areas of 250,000 to ...",Small-in a metro area with fewer than 1 millio...,Nonspecialized,0.462,0.060,0.185,0.084,0.216,0.178,0.059,0.100,0.149,0.174,13.099012,4.2,9.88,218,1069.0,1749.0,0.504,0.255,0.108,0.061876,0.208583,0.370259,0.359281,13.944628,6.173158


In [6]:
df.dtypes

row_id                                          int64
county_code                                    object
state                                          object
population                                    float64
renter_occupied_households                    float64
pct_renter_occupied                           float64
evictions                                     float64
rent_burden                                   float64
pct_white                                     float64
pct_af_am                                     float64
pct_hispanic                                  float64
pct_am_ind                                    float64
pct_asian                                     float64
pct_nh_pi                                     float64
pct_multiple                                  float64
pct_other                                     float64
poverty_rate                                  float64
rucc                                           object
urban_influence             

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562 entries, 0 to 1561
Data columns (total 45 columns):
row_id                                        1562 non-null int64
county_code                                   1562 non-null object
state                                         1562 non-null object
population                                    1562 non-null float64
renter_occupied_households                    1562 non-null float64
pct_renter_occupied                           1562 non-null float64
evictions                                     1235 non-null float64
rent_burden                                   1562 non-null float64
pct_white                                     1562 non-null float64
pct_af_am                                     1562 non-null float64
pct_hispanic                                  1562 non-null float64
pct_am_ind                                    1562 non-null float64
pct_asian                                     1562 non-null float64
pct_nh_pi    

In [8]:
df.describe()

,row_id,population,renter_occupied_households,pct_renter_occupied,evictions,rent_burden,pct_white,pct_af_am,pct_hispanic,pct_am_ind,pct_asian,pct_nh_pi,pct_multiple,pct_other,poverty_rate,pct_civilian_labor,pct_unemployment,pct_uninsured_adults,pct_uninsured_children,pct_adult_obesity,pct_adult_smoking,pct_diabetes,pct_low_birthweight,pct_excessive_drinking,pct_physical_inactivity,air_pollution_particulate_matter_value,homicides_per_100k,motor_vehicle_crash_deaths_per_100k,heart_disease_mortality_per_100k,pop_per_dentist,pop_per_primary_care_physician,pct_female,pct_below_18_years_of_age,pct_aged_65_years_and_older,pct_adults_less_than_a_high_school_diploma,pct_adults_with_high_school_diploma,pct_adults_with_some_college,pct_adults_bachelors_or_higher,birth_rate_per_1k,death_rate_per_1k
count,1562.000000,1.562000e+03,1.562000e+03,1562.000000,1235.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1560.000000,1560.000000,1560.000000,1344.000000,1560.000000,1446.000000,1100.000000,1560.000000,1542.000000,613.000000,1372.000000,1562.000000,1447.000000,1448.000000,1560.000000,1560.000000,1560.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000
mean,780.500000,1.083407e+05,1.490462e+04,28.525570,397.410526,28.537526,0.769045,0.088593,0.092012,0.018431,0.012709,0.000711,0.017583,0.000916,12.182780,0.470535,0.062551,0.220037,0.088844,0.304546,0.211682,0.106566,0.083326,0.164818,0.276981,11.637336,5.752414,21.715153,275.482714,3421.828611,2508.303867,0.499388,0.228672,0.167707,0.145666,0.346271,0.303030,0.205033,11.621356,10.415138
std,451.054875,3.745229e+05,6.255947e+04,8.121601,1522.801074,4.670467,0.203306,0.143505,0.141599,0.074762,0.026693,0.003059,0.015836,0.001657,5.783889,0.070936,0.022615,0.067500,0.041480,0.043550,0.064045,0.022521,0.021239,0.051483,0.053098,1.534144,4.297808,10.721369,57.827540,2538.670834,1960.312344,0.023933,0.034732,0.044555,0.067483,0.071152,0.052236,0.092001,2.756009,2.772070
min,0.000000,2.690000e+02,6.400000e+01,7.279000,-1.000000,9.909000,0.009547,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.186000,0.012000,0.053000,0.018000,0.133000,0.031000,0.033000,0.030000,0.032000,0.104000,7.209413,-0.080000,3.140000,76.000000,340.000000,279.000000,0.314000,0.082000,0.036000,0.019000,0.074297,0.114458,0.064128,3.654080,0.961076
25%,390.250000,1.045275e+04,1.078250e+03,22.874000,3.000000,25.869000,0.649967,0.005523,0.018747,0.001201,0.002391,0.000000,0.009465,0.000000,7.991000,0.425000,0.046000,0.171750,0.059000,0.283000,0.169000,0.092000,0.070000,0.129000,0.243000,10.432910,2.710000,14.047500,233.000000,1859.000000,1389.750000,0.495000,0.208000,0.140000,0.094226,0.300601,0.269461,0.142142,10.014818,8.613691
50%,780.500000,2.528200e+04,2.754000e+03,27.199500,27.000000,28.768000,0.846542,0.020599,0.037613,0.002610,0.005279,0.000000,0.014438,0.000302,11.173500,0.470500,0.061000,0.216000,0.079000,0.306000,0.206000,0.105000,0.080000,0.163000,0.281000,11.907300,4.540000,20.285000,270.000000,2730.000000,1969.500000,0.504000,0.227000,0.164000,0.129388,0.352176,0.303303,0.182365,11.435441,10.396898
75%,1170.750000,6.836150e+04,7.987000e+03,32.341500,162.500000,31.307250,0.933479,0.102047,0.092998,0.006121,0.011886,0.000403,0.020999,0.001197,15.018500,0.515000,0.076000,0.265000,0.109000,0.331250,0.249000,0.122000,0.091000,0.196000,0.312000,12.885680,7.600000,27.072500,311.000000,4064.500000,2840.000000,0.511000,0.246000,0.191000,0.187719,0.396556,0.337672,0.241234,12.940235,12.250655
max,1561.000000,1.002029e+07,1.760277e+06,73.008000,29251.000000,49.665000,0.995141,0.756011,0.987039,0.815817,0.418088,0.085037,0.184332,0.019582,38.792000,0.996000,0.242000,0.520000,0.327000,0.474000,0.513000,0.180000,0.182000,0.419000,0.446000,14.992477,26.920000,110.450000,511.000000,25169.000000,16740.000000,0.564000,0.415000,0.488000,0.535750,0.535536,0.477341,0.788153,29.034900,24

In [9]:
df.shape

(1562, 45)

### Remove features > 5% missing values and strings

In [10]:
df = df.drop(['row_id','county_code','state','evictions','pct_adult_smoking','pct_low_birthweight',
              'pct_excessive_drinking','homicides_per_100k','motor_vehicle_crash_deaths_per_100k',
              'pop_per_dentist','pop_per_primary_care_physician'],axis=1)

In [11]:
df

,population,renter_occupied_households,pct_renter_occupied,rent_burden,pct_white,pct_af_am,pct_hispanic,pct_am_ind,pct_asian,pct_nh_pi,pct_multiple,pct_other,poverty_rate,rucc,urban_influence,economic_typology,pct_civilian_labor,pct_unemployment,pct_uninsured_adults,pct_uninsured_children,pct_adult_obesity,pct_diabetes,pct_physical_inactivity,air_pollution_particulate_matter_value,heart_disease_mortality_per_100k,pct_female,pct_below_18_years_of_age,pct_aged_65_years_and_older,pct_adults_less_than_a_high_school_diploma,pct_adults_with_high_school_diploma,pct_adults_with_some_college,pct_adults_bachelors_or_higher,birth_rate_per_1k,death_rate_per_1k
0,3876.0,408.0,24.583,18.380,0.945945,0.010761,0.026038,0.005685,0.005635,0.000000,0.005935,0.000000,4.172,"Nonmetro - Completely rural or less than 2,500...",Noncore adjacent to micro area and does not co...,Manufacturing-dependent,0.550,0.023,0.107,0.062,0.310,0.100,0.342,11.022908,217,0.471,0.218,0.190,0.083250,0.327984,0.389168,0.199599,10.008982,9.752342
1,10224.0,1166.0,28.346,26.694,0.808959,0.002284,0.173426,0.001803,0.001493,0.000000,0.012033,0.000000,7.714,"Nonmetro - Completely rural or less than 2,500...",Micropolitan not adjacent to a metro area,Recreation,0.545,0.040,0.324,0.201,0.253,0.066,0.148,9.904099,181,0.472,0.290,0.077,0.099099,0.202202,0.302302,0.396396,14.482893,3.013219
2,27023.0,2927.0,21.641,31.028,0.956621,0.005141,0.013088,0.004728,0.004708,0.000000,0.014809,0.000905,9.789,"Nonmetro - Urban population of 2,500 to 19,999...",Noncore not adjacent to a metro/micro area and...,Recreation,0.508,0.063,0.129,0.032,0.296,0.087,0.258,11.011502,234,0.496,0.208,0.192,0.137725,0.387226,0.255489,0.219561,9.620346,10.873572
3,8735.0,1039.0,23.110,27.734,0.894835,0.000298,0.076576,0.000995,0.004297,0.000000,0.022999,0.000000,13.735,"Nonmetro - Urban population of 2,500 to 19,999...",Noncore not adjacent to a metro/micro area and...,Nonspecialized,0.545,0.038,0.207,0.084,0.255,0.104,0.274,12.845770,231,0.503,0.193,0.215,0.077154,0.308617,0.398798,0.215431,9.383225,10.870809
4,3681.0,365.0,21.985,19.673,0.923886,0.000496,0.042889,0.000000,0.013622,0.000000,0.016905,0.002202,8.217,"Nonmetro - Completely rural or less than 2,500...",Noncore adjacent to micro area and does not co...,Farm-dependent,0.474,0.034,0.177,0.075,0.324,0.113,0.340,11.565750,270,0.516,0.222,0.230,0.092537,0.362189,0.346269,0.199005,9.543286,14.723926
5,8540.0,751.0,22.543,34.937,0.560637,0.417547,0.000597,0.001294,0.000000,0.000000,0.019925,0.000000,21.051,Metro - Counties in metro areas of fewer than ...,Small-in a metro area with fewer than 1 millio...,Mining-dependent,0.361,0.110,0.246,0.100,0.322,0.148,0.317,12.822859,388,0.515,0.199,0.185,0.317000,0.388000,0.198000,0.097000,10.456274,14.139734
6,22712.0,1901.0,24.602,29.242,0.929799,0.005564,0.039416,0.002389,0.005106,0.000000,0.016234,0.001493,9.651,Metro - Counties in metro areas of 1 million p...,Large-in a metro area with at least 1 million ...,Recreation,0.470,0.061,0.258,0.126,0.275,0.108,0.231,13.395494,254,0.494,0.218,0.162,0.142572,0.287139,0.292124,0.278166,10.953481,6.818432
7,27974.0,3873.0,38.012,29.966,0.537084,0.008489,0.396900,0.017048,0.024534,0.002472,0.013473,0.000000,15.655,"Nonmetro - Urban population of 2,500 to 19,999...",Noncore adjacent to a small metro with town of...,Farm-dependent,0.462,0.106,0.297,0.117,0.267,0.078,0.188,8.035606,239,0.497,0.278,0.141,0.249004,0.293825,0.307769,0.149402,14.526719,9.660626
8,2027.0,295.0,55.572,21.808,0.184899,0.001981,0.010408,0.749382,0.000000,0.016572,0.036758,0.000000,36.120,"Nonmetro - Completely rural or less than 2,500...",Noncore not adjacent to a metro/micro area and...,Federal/State government-dependent,0.353,0.077,0.316,0.086,0.421,0.177,0.384,10.904799,511,0.500,0.415,0.075,0.181456,0.432702,0.291127,0.094716,26.816187,6.825939
9,17165.0,2223.0,29.769,28.664,0.961332,0.009077,0.011856,0.002779,0.000800,0.000000,0.014056,0.000100,17.793,"Nonmetro - Urban population of 2,500

### Treat Missing Values

In [12]:
df.isnull().sum()

population                                     0
renter_occupied_households                     0
pct_renter_occupied                            0
rent_burden                                    0
pct_white                                      0
pct_af_am                                      0
pct_hispanic                                   0
pct_am_ind                                     0
pct_asian                                      0
pct_nh_pi                                      0
pct_multiple                                   0
pct_other                                      0
poverty_rate                                   0
rucc                                           0
urban_influence                                0
economic_typology                              0
pct_civilian_labor                             0
pct_unemployment                               0
pct_uninsured_adults                           2
pct_uninsured_children                         2
pct_adult_obesity   

In [13]:
imputer = mdi.MeanMedianImputer(imputation_method='mean',variables=None)

In [14]:
imputer.fit(df)

MeanMedianImputer(imputation_method='mean',
                  variables=['population', 'renter_occupied_households',
                             'pct_renter_occupied', 'rent_burden', 'pct_white',
                             'pct_af_am', 'pct_hispanic', 'pct_am_ind',
                             'pct_asian', 'pct_nh_pi', 'pct_multiple',
                             'pct_other', 'poverty_rate', 'pct_civilian_labor',
                             'pct_unemployment', 'pct_uninsured_adults',
                             'pct_uninsured_children', 'pct_adult_o...
                             'pct_diabetes', 'pct_physical_inactivity',
                             'air_pollution_particulate_matter_value',
                             'heart_disease_mortality_per_100k', 'pct_female',
                             'pct_below_18_years_of_age',
                             'pct_aged_65_years_and_older',
                             'pct_adults_less_than_a_high_school_diploma',
                     

In [15]:
df1 = imputer.transform(df)

In [16]:
df1.isnull().sum()

population                                    0
renter_occupied_households                    0
pct_renter_occupied                           0
rent_burden                                   0
pct_white                                     0
pct_af_am                                     0
pct_hispanic                                  0
pct_am_ind                                    0
pct_asian                                     0
pct_nh_pi                                     0
pct_multiple                                  0
pct_other                                     0
poverty_rate                                  0
rucc                                          0
urban_influence                               0
economic_typology                             0
pct_civilian_labor                            0
pct_unemployment                              0
pct_uninsured_adults                          0
pct_uninsured_children                        0
pct_adult_obesity                       

In [17]:
df1.shape

(1562, 34)

### Save a copy

In [18]:
#df.to_csv('trainfilled2.csv',index=False)